In [21]:
import json

# Load the old JSON data
with open('yolo.json', 'r') as f:
    old_data = json.load(f)

# Create a new JSON object
new_data = {
    'frames': [],
    'class': old_data[0]['class'],
    'filename': old_data[0]['filename']
}

# Iterate through each frame in the old data
for frame in old_data[0]['frames']:
    # Create a new frame object
    new_frame = {
        'timestamp': frame['timestamp'],
        'num': frame['num'],
        'class': frame['class'],
        'hypotheses': []
    }
    
    # Keep only one entry in the hypotheses list
    if len(frame['hypotheses']) > 0:
        new_frame['hypotheses'].append(frame['hypotheses'][0])
    
    # Add the new frame to the new data
    new_data['frames'].append(new_frame)

# Save the new JSON data to a file
with open('new_data.json', 'w') as f:
    json.dump([new_data], f, indent=4)




In [23]:
#using mot metrics package

import motmetrics as mm
import json

# Load ground truth and DeepSORT JSON files
with open('gt.json', 'r') as f:
    gt_data = json.load(f)
with open('yolo.json', 'r') as f:
    ds_data = json.load(f)

# Create an accumulator that will be updated during each frame
acc = mm.MOTAccumulator(auto_id=True)

# Iterate through each frame of data
for frame in range(len(gt_data[0]['frames'])):
    # Get ground truth and DeepSORT data for the current frame
    gt_frame = gt_data[0]['frames'][frame]
    ds_frame = ds_data[0]['frames'][frame]


    # Get the ground truth and DeepSORT bounding boxes for the current frame
    #gt_boxes = [d['bbox'] for d in gt_frame['annotations']]
    #ds_boxes = [d['bbox'] for d in ds_frame['hypotheses']]

    gt_boxes = [[d['x'], d['y'], d['width'], d['height']] for d in gt_frame['annotations']]           
    ds_boxes = [[d['x'], d['y'], d['width'], d['height']] for d in ds_frame['hypotheses']]

    # Calculate the intersection over union (IOU) distance between ground truth and DeepSORT bounding boxes
    distances = mm.distances.iou_matrix(gt_boxes, ds_boxes, max_iou=0.5)

    # Update the accumulator with the calculated distances
    acc.update(
        [d['id'] for d in gt_frame['annotations']],
        [d['id'] for d in ds_frame['hypotheses']],
        distances
    )

# Calculate the MOTA and MOTP metrics from the accumulator data
mh = mm.metrics.create()
summary = mh.compute(acc, metrics=['mota', 'motp'], name='acc')
print(summary)


     mota  motp
acc   0.0   NaN


C:\Users\bhava\.conda\envs\carla-sim\lib\site-packages\motmetrics\metrics.py:290: RuntimeWarning: invalid value encountered in scalar divide
  return df.noraw['D'].sum() / num_detections


In [24]:
# general script 

import json
from collections import defaultdict
import numpy as np

# Load ground truth and DeepSORT JSON files
with open('gt.json', 'r') as f:
    gt_data = json.load(f)
with open('new_data.json', 'r') as f:
    ds_data = json.load(f)

# Initialize variables for calculating MOTA and MOTP metrics
num_matches = 0
num_misses = 0
num_false_positives = 0
num_mismatches = 0
total_distance = 0

# Iterate through each frame of data
for frame in range(len(gt_data[0]['frames'])):
    # Get ground truth and DeepSORT data for the current frame
    gt_frame = gt_data[0]['frames'][frame]
    ds_frame = ds_data[0]['frames'][frame]

    # Get the ground truth and DeepSORT bounding boxes for the current frame
    gt_boxes = [[d['x'], d['y'], d['width'], d['height']] for d in gt_frame['annotations']]
    ds_boxes = [[d['x'], d['y'], d['width'], d['height']] for d in ds_frame['hypotheses']]

    # Calculate the intersection over union (IOU) between ground truth and DeepSORT bounding boxes
    iou_matrix = []
    for gt_box in gt_boxes:
        iou_row = []
        for ds_box in ds_boxes:
            x1, y1, w1, h1 = gt_box
            x2, y2, w2, h2 = ds_box
            x_left = max(x1, x2)
            y_top = max(y1, y2)
            x_right = min(x1 + w1, x2 + w2)
            y_bottom = min(y1 + h1, y2 + h2)
            if x_right < x_left or y_bottom < y_top:
                iou_row.append(0.0)
                continue
            intersection_area = (x_right - x_left) * (y_bottom - y_top)
            bb1_area = w1 * h1
            bb2_area = w2 * h2
            iou_row.append(intersection_area / float(bb1_area + bb2_area - intersection_area))
        iou_matrix.append(iou_row)

    # Find matches between ground truth and DeepSORT bounding boxes using the Hungarian algorithm
    if len(gt_boxes) >0 and len(ds_boxes) > 0:
        
        from scipy.optimize import linear_sum_assignment
        row_ind, col_ind = linear_sum_assignment(1 - np.array(iou_matrix))
        matches = [(row_ind[i], col_ind[i]) for i in range(len(row_ind)) if iou_matrix[row_ind[i]][col_ind[i]] > 0.5]
    else:
        matches= []
    # Update MOTA and MOTP metrics with the current frame data
    num_matches += len(matches)
    num_misses += len(gt_boxes) - len(matches)
    num_false_positives += len(ds_boxes) - len(matches)
    total_distance += sum([iou_matrix[i][j] for i, j in matches])


# Calculate the final MOTA and MOTP values
mota = 1 - (num_misses + num_false_positives + num_mismatches) / sum([len(frame['annotations']) for frame in gt_data[0]['frames']])
motp = total_distance / num_matches

print(f'MOTA: {mota}')
print(f'MOTP: {motp}')


IndexError: list index out of range